In [ ]:
using LinearAlgebra, TensorOperations, Printf

function build_ising_hamiltonian()
    X = [0 1; 1 0]
    Z = [1 0; 0 -1]
    Id = I(2)
    h = -kron(Z, Z) - 0.5 * (kron(X, Id) + kron(Id, X))
    return h
end

# 2. ランダムなMERAテンソル（Isometry w, Disentangler u）の生成
function random_mera_tensors(chi_in, chi_out)
    # Disentangler u: (chi_in * chi_in) -> (chi_in * chi_in) のユニタリ
    d_u = chi_in * chi_in
    Q_u, R_u = qr(randn(d_u, d_u))
    u = Matrix(Q_u)

    # Isometry w: (chi_in) -> (chi_out * chi_out) の等長写像
    # ここでは逆向き（粗視化）: (chi_in * chi_in) -> (chi_out) と考える
    # つまり 2サイトを1サイトに圧縮する
    d_in = chi_in * chi_in
    d_out = chi_out
    Q_w, R_w = qr(randn(d_in, d_in))
    w = Matrix(Q_w)[:, 1:d_out] # 列を取り出して等長写像にする
    return u, w
end

# --- 3. くりこみステップ (Descending Superoperator) ---

"""
Hamiltonian Coarse-Graining (Real-space RG step)
Binary MERAの簡易版スキーム
入力:
  h_bottom: 下層のハミルトニアン (2サイト相互作用, 足は [i, j, k, l])
  u: Disentangler
  w: Isometry
出力:
  h_top: 粗視化された上層のハミルトニアン
"""
function coarse_grain_hamiltonian(h_bottom, u, w, chi)
    # テンソル演算のためにreshape
    # h: (chi, chi, chi, chi)  (下図の足: 左下, 右下, 左上, 右上)
    h_tensor = reshape(h_bottom, (chi, chi, chi, chi))

    # u: (chi, chi, chi, chi)
    u_tensor = reshape(u, (chi, chi, chi, chi))

    # w: (chi*chi, chi_new) -> reshape -> (chi, chi, chi_new)
    w_tensor = reshape(w, (chi, chi, chi)) # (left_in, right_in, top_out)

    # --- 縮約 (Tensor Contraction) ---
    # 物理的なイメージ:
    # 1. Disentangler u で絡まりを解く
    # 2. Hamiltonian h が作用
    # 3. Isometry w で上のレイヤーへ持ち上げる

    # ここでは「Binary MERA」の相互作用のパスの一つ（Left-Interaction）を計算
    # Note: 実際には3種類の項（Left, Center, Right）を足し合わせる必要がありますが、
    # 構造理解のために中心的な項のみ計算します。

    @tensor begin
        # 1. U^\dagger H U
        # h_tilde[a,b, c,d] := u'[a,b, i,j] * h_tensor[i,j, k,l] * u[k,l, c,d]
        # (随伴は複素共役)
        h_tilde[a, b, c, d] := conj(u_tensor[i, j, a, b]) * h_tensor[i, j, k, l] * u_tensor[k, l, c, d]

        # 2. W^\dagger H_tilde W
        # 粗視化: 下の4本足(a,b,c,d)を、wを使って上の2本足(top1, top2)にまとめる
        # h_new[t1, t2] ... ではなく、ハミルトニアンは2体相互作用を保つ形にするのが一般的
        # ですが、ここでは「2サイト -> 1サイト」へのマッピングとして
        # エネルギー密度がどう変わるかを見る。
    end

    # 簡単のため、Average Hamiltonian (Heff) を算出する形にする
    # h_eff[t1, t2] := w'[a,b, t1] * h_tilde[a,b, c,d] * w[c,d, t2]
    @tensor begin
        h_top[x, y] := conj(w_tensor[a, b, x]) * h_tilde[a, b, c, d] * w_tensor[c, d, y]
    end

    return h_top
end

# --- 実行してみる ---

chi = 2 # スピン1/2
h_init = build_ising_hamiltonian()
println("Initial Energy Norm: ", norm(h_init))

# ランダムなMERA層で1回くりこんでみる
# （最適化されていないのでエネルギーは下がらないが、次元が変わる様子を確認）
u, w = random_mera_tensors(chi, chi)

h_renorm = coarse_grain_hamiltonian(h_init, u, w, chi)

println("Renormalized Hamiltonian (1-site effective field):")
display(h_renorm)
println("\nNorm after RG step: ", norm(h_renorm))


Initial Energy Norm: 2.449489742783178
Renormalized Hamiltonian (1-site effective field):


2×2 Matrix{Float64}:
  1.04014   -0.359058
 -0.359058   0.511473


Norm after RG step: 1.265440214620909
